In [1]:
# Jessica Gallo
# CSC767 Neural Networks & Deep Learning
# Project 2
# Part E

In [2]:
# This is the original code with Adadelta
# This was run with the original dataset under limited data files:
# 0.txt, 6.txt and 8.txt.

In [3]:
#! /usr/bin/env python
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from helper import data_helpers
from text_cnn.LW_text_cnn import TextCNN
from tensorflow.contrib import learn
import yaml
import math
import sys

C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [4]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

In [5]:
# Parameters
# ==================================================

In [6]:
# Data loading params
tf.compat.v1.flags.DEFINE_float("dev_sample_percentage", 0.2, "Percentage of the training data to use for validation")

In [7]:
# Model Hyperparameters
tf.compat.v1.flags.DEFINE_boolean("enable_word_embeddings", False, "Enable/disable the word embedding (default: True)")
tf.compat.v1.flags.DEFINE_integer("embedding_dim", 200, "Dimensionality of character embedding (default: 128)")
tf.compat.v1.flags.DEFINE_string("filter_sizes", "2,3,5", "Comma-separated filter sizes (default: '2,3,5')")
tf.compat.v1.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 120)")
tf.compat.v1.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.compat.v1.flags.DEFINE_float("l2_reg_lambda", 0.0001, "L2 regularization lambda (default: 0.0)")

In [8]:
# Training parameters
tf.compat.v1.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.compat.v1.flags.DEFINE_integer("num_epochs",10, "Number of training epochs (default: 10)")
tf.compat.v1.flags.DEFINE_integer("evaluate_every", 50, "Evaluate model on dev set after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("checkpoint_every", 50, "Save model after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [9]:
# Misc Parameters
tf.compat.v1.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.compat.v1.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.compat.v1.flags.DEFINE_float("decay_coefficient", 2.5, "Decay coefficient (default: 2.5)")

In [10]:
FLAGS = tf.compat.v1.flags.FLAGS

In [11]:
YML_PATH = os.path.join(ROOT_DIR, "train/helper/config.yml")
print("***********************YML_PATH", YML_PATH)

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml


In [12]:
with open(YML_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [13]:
# ===================================================================================
# added this because of 'Unrecognizable flag error: Unknown command line flag f'
tf.compat.v1.flags.DEFINE_string('f','','')
# ===================================================================================

dataset_name = cfg["datasets"]["default"]
if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
    embedding_name = cfg['word_embeddings']['default']
    embedding_dimension = cfg['word_embeddings'][embedding_name]['dimension']
else:
    embedding_dimension = FLAGS.embedding_dim

In [14]:
# Data Preparation
# ==================================================

# Load data
print("Loading data...")
datasets = None
if dataset_name == "tobacco":
    data_path = os.path.join(ROOT_DIR, cfg["datasets"][dataset_name]['parent_dir'] +'/')
    print("*************data_path", data_path)    
    datasets = data_helpers.get_datasets_tobacco(data_path)

elif dataset_name == "localdata":
    datasets = data_helpers.get_datasets_localdata(container_path=cfg["datasets"][dataset_name]["container_path"],
                                                     categories=cfg["datasets"][dataset_name]["categories"],
                                                     shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                     random_state=cfg["datasets"][dataset_name]["random_state"])

x_text, y = data_helpers.load_data_labels(datasets)

Loading data...
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']


In [15]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 16099
Train/Dev split: 9549/2387


In [16]:
# Training
# ==================================================

In [17]:
# ===========================================================================
# Added to fix error
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# ===========================================================================

with tf.Graph().as_default():
    session_conf = tf.compat.v1.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)

    sess = tf.compat.v1.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dimension,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
       
        global_step = tf.compat.v1.Variable(0, name="global_step", trainable=False)
        # ===========================================================================================
        # Optimizers
        early_train_op= tf.compat.v1.train.AdamOptimizer(cnn.learning_rate)
        # late_train_op = tf.compat.v1.train.MomentumOptimizer(cnn.learning_rate,0.9,use_nesterov=True) 
        late_train_op = tf.compat.v1.train.AdadeltaOptimizer(cnn.learning_rate, rho=0.95) 
        # ===========================================================================================
        
        grads_and_vars_1 = early_train_op.compute_gradients(cnn.loss)
        train_op_1 = early_train_op.apply_gradients(grads_and_vars_1, global_step=global_step)
        
        grads_and_vars_2 = late_train_op.compute_gradients(cnn.loss)
        train_op_2 = late_train_op.apply_gradients(grads_and_vars_2, global_step=global_step)  

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.compat.v1.summary.scalar("loss", cnn.loss)
        acc_summary = tf.compat.v1.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.compat.v1.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.compat.v1.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.compat.v1.train.Saver(tf.compat.v1.trainable_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.compat.v1.global_variables_initializer())
        if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
            vocabulary = vocab_processor.vocabulary_
            initW = None
            if embedding_name == 'word2vec':
                # load embedding vectors from the word2vec
                print("Load word2vec file {}".format(cfg['word_embeddings']['word2vec']['path']))
                initW = data_helpers.load_embedding_vectors_word2vec(vocabulary,
                                                                     cfg['word_embeddings']['word2vec']['path'],
                                                                     cfg['word_embeddings']['word2vec']['binary'])
                print("word2vec file has been loaded")
            elif embedding_name == 'glove':
                # load embedding vectors from the glove
                print("Load glove file {}".format(cfg['word_embeddings']['glove']['path']))
                initW = data_helpers.load_embedding_vectors_glove(vocabulary,
                                                                  cfg['word_embeddings']['glove']['path'],
                                                                  embedding_dimension)
                print("glove file has been loaded\n")
            sess.run(cnn.W.assign(initW))

        def train_step(x_batch, y_batch, learning_rate):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob,                
              cnn.learning_rate: learning_rate              
            }
            
            #Conditional optimizer switch
            train_op = train_op_1 if tf.compat.v1.train.global_step(sess, global_step) <=(train_input_size/FLAGS.batch_size)*3 else train_op_2
                        
            _, step, summaries, loss, accuracy = sess.run(                
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],                
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            
            print("{}: step {}, loss {:g}, acc {:g}, learning_rate {:g}"
                  .format(time_str, step, loss, accuracy, learning_rate))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                

        train_input_size=len(list(zip(x_train, y_train)))
        print('Trainning input set: x_train, y_train',train_input_size)
        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        
        # It uses dynamic learning rate with a high value at the beginning to speed up the training
        max_learning_rate = 0.004
        min_learning_rate = 0.0001        
        decay_speed = FLAGS.decay_coefficient*len(y_train)/FLAGS.batch_size
        
        # Training loop. For each batch...
        counter = 0
        print("*********Trainable PARAMETERS***********",np.sum([np.prod(v.get_shape().as_list()) for v in 
                                                                 tf.compat.v1.trainable_variables()]))
        for batch in batches:
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-counter/decay_speed)
            counter += 1
            x_batch, y_batch = zip(*batch)      
            
            if tf.compat.v1.train.global_step(sess, global_step) == ((train_input_size/FLAGS.batch_size)*3)+1: ##(2534*3)+1:
                min_learning_rate = 0.0035
                
            train_step(x_batch, y_batch, learning_rate)
            
            current_step = tf.compat.v1.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
(?, 41, 1, 1)
(?, 40, 1, 1)
(?, 42, 200, 1)
(?, 40, 1, 1)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Writing to C:\Users\User\Lightweighted-CNN-for-Document-Classification\train\runs\1607550540

Trainning input set: x_train, y_train 9549
*********Trainable PARAMETERS*********** 3224697
2020-12-09T16:49:04.503350: step 1, loss 1.29593, acc 0.3125, learning_rate 0.004
2020-12-09T16:49:05.143320: step 2, loss 1.10861, acc 0.375, learning_rate 0.00399478
2020-12-09T16:49:05.743782: step 3, loss 1.08022, acc 0.59375, learning_rate 0.00398956
2020-12-09T16:49:06.347282: step 4, loss 1.46448, acc 0.375, learning_rate 0.00398435
2020-12-09T16:49:06.959282: step 5, loss 1.06581, acc 0.4375, learning_rat

2020-12-09T16:49:55.581416: step 80, loss 0.875997, acc 0.59375, learning_rate 0.00360812
2020-12-09T16:49:56.155390: step 81, loss 0.812456, acc 0.53125, learning_rate 0.00360342
2020-12-09T16:49:56.737889: step 82, loss 1.12217, acc 0.59375, learning_rate 0.00359873
2020-12-09T16:49:57.337164: step 83, loss 0.93291, acc 0.4375, learning_rate 0.00359404
2020-12-09T16:49:57.932203: step 84, loss 0.939182, acc 0.5, learning_rate 0.00358936
2020-12-09T16:49:58.521288: step 85, loss 0.844904, acc 0.53125, learning_rate 0.00358469
2020-12-09T16:49:59.103100: step 86, loss 0.750121, acc 0.59375, learning_rate 0.00358002
2020-12-09T16:49:59.679135: step 87, loss 0.744339, acc 0.65625, learning_rate 0.00357536
2020-12-09T16:50:00.262602: step 88, loss 0.678971, acc 0.78125, learning_rate 0.0035707
2020-12-09T16:50:00.832598: step 89, loss 0.840856, acc 0.59375, learning_rate 0.00356605
2020-12-09T16:50:01.392658: step 90, loss 0.59986, acc 0.75, learning_rate 0.00356141
2020-12-09T16:50:01.94

2020-12-09T16:50:54.761923: step 167, loss 0.753398, acc 0.65625, learning_rate 0.00322196
2020-12-09T16:50:55.342423: step 168, loss 0.703597, acc 0.6875, learning_rate 0.00321778
2020-12-09T16:50:55.914316: step 169, loss 0.65576, acc 0.75, learning_rate 0.0032136
2020-12-09T16:50:56.480807: step 170, loss 0.675519, acc 0.71875, learning_rate 0.00320943
2020-12-09T16:50:57.073723: step 171, loss 0.802467, acc 0.59375, learning_rate 0.00320527
2020-12-09T16:50:57.644226: step 172, loss 0.764075, acc 0.75, learning_rate 0.00320111
2020-12-09T16:50:58.282137: step 173, loss 0.569573, acc 0.8125, learning_rate 0.00319695
2020-12-09T16:50:58.862709: step 174, loss 0.658978, acc 0.65625, learning_rate 0.0031928
2020-12-09T16:50:59.434287: step 175, loss 0.672293, acc 0.6875, learning_rate 0.00318866
2020-12-09T16:51:00.013785: step 176, loss 0.541843, acc 0.78125, learning_rate 0.00318452
2020-12-09T16:51:00.598993: step 177, loss 0.83123, acc 0.625, learning_rate 0.00318039
2020-12-09T16:

2020-12-09T16:51:54.303334: step 254, loss 0.674201, acc 0.65625, learning_rate 0.00287831
2020-12-09T16:51:54.880236: step 255, loss 0.734341, acc 0.65625, learning_rate 0.00287458
2020-12-09T16:51:55.471762: step 256, loss 0.761723, acc 0.75, learning_rate 0.00287087
2020-12-09T16:51:56.049268: step 257, loss 0.656903, acc 0.65625, learning_rate 0.00286716
2020-12-09T16:51:56.635519: step 258, loss 0.712284, acc 0.75, learning_rate 0.00286345
2020-12-09T16:51:57.223507: step 259, loss 0.686968, acc 0.625, learning_rate 0.00285975
2020-12-09T16:51:57.798086: step 260, loss 0.788221, acc 0.59375, learning_rate 0.00285605
2020-12-09T16:51:58.387599: step 261, loss 0.834849, acc 0.625, learning_rate 0.00285236
2020-12-09T16:51:58.975639: step 262, loss 0.756999, acc 0.5625, learning_rate 0.00284867
2020-12-09T16:51:59.558610: step 263, loss 0.735828, acc 0.625, learning_rate 0.00284499
2020-12-09T16:52:00.145608: step 264, loss 0.684606, acc 0.71875, learning_rate 0.00284131
2020-12-09T1

2020-12-09T16:52:48.595705: step 340, loss 0.52452, acc 0.78125, learning_rate 0.0025758
2020-12-09T16:52:49.183957: step 341, loss 0.600264, acc 0.71875, learning_rate 0.00257248
2020-12-09T16:52:49.792335: step 342, loss 0.454243, acc 0.8125, learning_rate 0.00256917
2020-12-09T16:52:50.378330: step 343, loss 0.434903, acc 0.84375, learning_rate 0.00256586
2020-12-09T16:52:50.961831: step 344, loss 0.323872, acc 0.875, learning_rate 0.00256256
2020-12-09T16:52:51.559817: step 345, loss 0.461886, acc 0.8125, learning_rate 0.00255926
2020-12-09T16:52:52.137165: step 346, loss 0.477969, acc 0.78125, learning_rate 0.00255596
2020-12-09T16:52:52.705055: step 347, loss 0.54999, acc 0.71875, learning_rate 0.00255268
2020-12-09T16:52:53.296670: step 348, loss 0.383958, acc 0.84375, learning_rate 0.00254939
2020-12-09T16:52:53.894157: step 349, loss 0.567105, acc 0.78125, learning_rate 0.00254611
2020-12-09T16:52:54.470548: step 350, loss 0.390768, acc 0.84375, learning_rate 0.00254283

Evalu

...

2020-12-09T17:19:02.214803: step 2582, loss 0.150824, acc 0.90625, learning_rate 0.000222611
2020-12-09T17:19:03.040304: step 2583, loss 0.166162, acc 0.90625, learning_rate 0.000222447
2020-12-09T17:19:03.804806: step 2584, loss 0.120558, acc 1, learning_rate 0.000222283
2020-12-09T17:19:04.764306: step 2585, loss 0.18928, acc 0.9375, learning_rate 0.000222119
2020-12-09T17:19:05.654310: step 2586, loss 0.135714, acc 0.96875, learning_rate 0.000221956
2020-12-09T17:19:06.516305: step 2587, loss 0.178912, acc 0.9375, learning_rate 0.000221792
2020-12-09T17:19:07.424304: step 2588, loss 0.23465, acc 0.875, learning_rate 0.000221629
2020-12-09T17:19:08.373803: step 2589, loss 0.124697, acc 0.9375, learning_rate 0.000221466
2020-12-09T17:19:09.261837: step 2590, loss 0.272896, acc 0.875, learning_rate 0.000221303
2020-12-09T17:19:10.173307: step 2591, loss 0.191326, acc 0.90625, learning_rate 0.000221141
2020-12-09T17:19:11.109848: step 2592, loss 0.151651, acc 0.96875, learning_rate 0.00

2020-12-09T17:20:02.317399: step 2667, loss 0.461965, acc 0.78125, learning_rate 0.000209408
2020-12-09T17:20:02.877810: step 2668, loss 0.244043, acc 0.90625, learning_rate 0.000209261
2020-12-09T17:20:03.423705: step 2669, loss 0.247304, acc 0.90625, learning_rate 0.000209115
2020-12-09T17:20:03.979472: step 2670, loss 0.159107, acc 0.9375, learning_rate 0.000208968
2020-12-09T17:20:04.537216: step 2671, loss 0.107272, acc 0.96875, learning_rate 0.000208823
2020-12-09T17:20:05.079687: step 2672, loss 0.200441, acc 0.875, learning_rate 0.000208677
2020-12-09T17:20:05.644205: step 2673, loss 0.323558, acc 0.84375, learning_rate 0.000208531
2020-12-09T17:20:06.189736: step 2674, loss 0.300995, acc 0.875, learning_rate 0.000208386
2020-12-09T17:20:06.742236: step 2675, loss 0.119707, acc 0.96875, learning_rate 0.000208241
2020-12-09T17:20:07.312920: step 2676, loss 0.135605, acc 0.96875, learning_rate 0.000208096
2020-12-09T17:20:07.878903: step 2677, loss 0.157674, acc 0.96875, learning

2020-12-09T17:20:58.741128: step 2752, loss 0.218854, acc 0.875, learning_rate 0.000197626
2020-12-09T17:20:59.304025: step 2753, loss 0.190196, acc 0.9375, learning_rate 0.000197495
2020-12-09T17:20:59.860115: step 2754, loss 0.317911, acc 0.875, learning_rate 0.000197364
2020-12-09T17:21:00.402176: step 2755, loss 0.108527, acc 0.96875, learning_rate 0.000197234
2020-12-09T17:21:00.956174: step 2756, loss 0.224075, acc 0.9375, learning_rate 0.000197104
2020-12-09T17:21:01.514173: step 2757, loss 0.259599, acc 0.90625, learning_rate 0.000196974
2020-12-09T17:21:02.076597: step 2758, loss 0.242463, acc 0.90625, learning_rate 0.000196844
2020-12-09T17:21:02.634356: step 2759, loss 0.370637, acc 0.875, learning_rate 0.000196714
2020-12-09T17:21:03.298571: step 2760, loss 0.229667, acc 0.90625, learning_rate 0.000196584
2020-12-09T17:21:04.080609: step 2761, loss 0.447801, acc 0.84375, learning_rate 0.000196455
2020-12-09T17:21:04.743002: step 2762, loss 0.168072, acc 0.9375, learning_rat

2020-12-09T17:21:54.443997: step 2840, loss 0.185344, acc 0.90625, learning_rate 0.000186763
2020-12-09T17:21:54.991497: step 2841, loss 0.187532, acc 0.9375, learning_rate 0.000186647
2020-12-09T17:21:55.546699: step 2842, loss 0.059085, acc 0.96875, learning_rate 0.000186531
2020-12-09T17:21:56.103100: step 2843, loss 0.251168, acc 0.9375, learning_rate 0.000186415
2020-12-09T17:21:56.666052: step 2844, loss 0.312737, acc 0.875, learning_rate 0.000186299
2020-12-09T17:21:57.228013: step 2845, loss 0.328907, acc 0.78125, learning_rate 0.000186184
2020-12-09T17:21:57.809019: step 2846, loss 0.310713, acc 0.84375, learning_rate 0.000186068
2020-12-09T17:21:58.383019: step 2847, loss 0.31178, acc 0.90625, learning_rate 0.000185953
2020-12-09T17:21:58.942073: step 2848, loss 0.323198, acc 0.84375, learning_rate 0.000185838
2020-12-09T17:21:59.482111: step 2849, loss 0.385643, acc 0.8125, learning_rate 0.000185723
2020-12-09T17:22:00.055491: step 2850, loss 0.202576, acc 0.90625, learning_

2020-12-09T17:22:51.525906: step 2925, loss 0.173928, acc 0.90625, learning_rate 0.00017742
2020-12-09T17:22:52.091614: step 2926, loss 0.33863, acc 0.84375, learning_rate 0.000177316
2020-12-09T17:22:52.654820: step 2927, loss 0.17663, acc 0.9375, learning_rate 0.000177213
2020-12-09T17:22:53.202624: step 2928, loss 0.20038, acc 0.875, learning_rate 0.000177109
2020-12-09T17:22:53.771115: step 2929, loss 0.28523, acc 0.875, learning_rate 0.000177006
2020-12-09T17:22:54.332715: step 2930, loss 0.22428, acc 0.9375, learning_rate 0.000176903
2020-12-09T17:22:54.892715: step 2931, loss 0.18864, acc 0.90625, learning_rate 0.0001768
2020-12-09T17:22:55.446834: step 2932, loss 0.188744, acc 0.96875, learning_rate 0.000176697
2020-12-09T17:22:56.010435: step 2933, loss 0.152655, acc 0.96875, learning_rate 0.000176594
2020-12-09T17:22:56.577442: step 2934, loss 0.227432, acc 0.875, learning_rate 0.000176491
2020-12-09T17:22:57.127861: step 2935, loss 0.161167, acc 0.96875, learning_rate 0.0001

In [18]:
# ===========================================================================
# Eval.py
# ===========================================================================

In [20]:
!python eval_SGD_small.py --eval_train --checkpoint_dir="./runs/1607550540/checkpoints/"

***********************YML_PATH C:\Users\User\Lightweighted-CNN-for-Document-Classification\train/helper/config.yml
dataset_name:  tobacco
Loading data !
*************data_path C:\Users\User\Lightweighted-CNN-for-Document-Classification\data/tobacco-data/
['0.txt', '6.txt', '8.txt']
Total number of test examples: 11936
Vocabulary Size: 16099
Train/Dev split: 9549/2387
size of x_dev, y_dev: 2387 2387

Evaluating...



C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (